# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |

## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.

In [1]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client= OpenAI(api_key=OPENAI_API_KEY)

text = '안녕하세요!'

response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = [text]
)
print(response.data[0].embedding)

[-0.010550308972597122, -0.05413339287042618, -0.03419528529047966, 0.041170332580804825, 0.01979553885757923, -0.04191609099507332, -0.029347844421863556, 0.05654614791274071, -0.031694795936346054, -0.0525980070233345, -0.04288119450211525, -0.01726214773952961, 0.017733732238411903, -0.016275113448500633, -0.01822724938392639, 0.048518259078264236, -0.060011737048625946, 0.014662954956293106, -0.027944061905145645, 0.02410559169948101, 0.05970465764403343, 0.0032928588334470987, -0.007013433147221804, -0.020990947261452675, 0.057598985731601715, 0.06501271575689316, 0.038362767547369, 0.027110565453767776, 0.005124357528984547, -0.03829696401953697, 0.02298695221543312, -0.021396728232502937, -0.011822487227618694, -0.03408561274409294, -0.0036520299036055803, 0.039152394980192184, -0.0002306509268237278, -0.03202380612492561, -0.051939982920885086, -0.03788021579384804, -0.02842661179602146, -0.006635069381445646, -0.01574869453907013, 0.0356648713350296, 0.03520425409078598, 0.017

In [26]:
type(response)

openai.types.create_embedding_response.CreateEmbeddingResponse

In [6]:
def texts_to_embedding(texts, model='text-embedding-3-small'):
  # 특수문자/개행문자등을 제거하면 임베딩 품질 높아진다.
  texts = [text.replace('\n', ' ') for text in texts]
  response = client.embeddings.create(
      model=model,
      input=texts
  )
  return [data.embedding for data in response.data]

output = texts_to_embedding(['hello world', 'byebye world'])

import numpy as np
np.array(output).shape

(2, 1536)

In [28]:
type(output)

list

## 음식리뷰 유사도 검색

In [11]:
!gdown 1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD

Downloading...
From: https://drive.google.com/uc?id=1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD
To: /content/fine_food_reviews_1k.csv
100% 439k/439k [00:00<00:00, 84.4MB/s]


In [13]:
# 데이터 로드
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens
Unnamed: 0,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
 6   n_tokens   1000 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 62.5+ KB


In [16]:
# Summary + Text -> Combined
df['combined'] = df['Summary'].str.strip() + ';/ ' + df['Text'].str.strip()
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combied,combined
Unnamed: 0,,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...,where does one start...and stop... with a tre...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces;/ Not pleased at all. When I...,Arrived in pieces;/ Not pleased at all. When I...
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .;/ I'...","It isn't blanc mange, but isn't bad . . .;/ I'..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.;/ ...,These also have SALT and it's not sea salt.;/ ...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product;/ My dog was suffering ...,Happy with the product;/ My dog was suffering ...


In [19]:
# 임베딩변환
df['embedding'] = texts_to_embedding(df['combined'].tolist())
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combied,combined,embedding
Unnamed: 0,,,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...,where does one start...and stop... with a tre...,"[0.029458723962306976, -0.021374186500906944, ..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces;/ Not pleased at all. When I...,Arrived in pieces;/ Not pleased at all. When I...,"[0.007512290962040424, 0.03292679786682129, -0..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .;/ I'...","It isn't blanc mange, but isn't bad . . .;/ I'...","[3.479194492683746e-05, 0.004105150233954191, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.;/ ...,These also have SALT and it's not sea salt.;/ ...,"[-0.016487499698996544, 0.01567600667476654, -..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product;/ My dog was suffering ...,Happy with the product;/ My dog was suffering ...,"[0.0015896736877039075, -0.07027246803045273, ..."


In [20]:
# embed_df로 변환
embed_df = df[['embedding']]
# embed_df = df['embedding'].to_frame()
embed_df.index = df['combined']
embed_df.head()

,embedding
combined,
where does one start...and stop... with a treat like this;/ Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.029458723962306976, -0.021374186500906944, ..."
"Arrived in pieces;/ Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.007512290962040424, 0.03292679786682129, -0..."
"It isn't blanc mange, but isn't bad . . .;/ I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[3.479194492683746e-05, 0.004105150233954191, ..."
"These also have SALT and it's not sea salt.;/ I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.016487499698996544, 0.01567600667476654, -..."
Happy with the product;/ My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[0.0015896736877039075, -0.07027246803045273, ..."


In [25]:
# 유사도 검색
from sklearn.metrics.pairwise import cosine_similarity

def review_search(query, emb_df, top_n=5):
    # 검색어 동일한 임베딩변환
    query_emb = texts_to_embedding([query])

    # cosine_similarity([emb], query_emb)는 shape (1, 1) 행렬을 리턴
    # cosine_similarity()는 DataFrame을 반환하지 않고 항상 numpy.ndarray를 반환합니다.
    # [0, 0]을 통해 스칼라 유사도 값만 추출
    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])
    top_n_texts = embed_df.sort_values('cos_sim', ascending=False).head(top_n)
    return top_n_texts

review_search('best bread', embed_df)

/tmp/ipython-input-25-3659971228.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


,embedding,cos_sim
combined,,
GOOD GLUTEN FREE BREAD STCK MIX;/ Makes very good break sticks.. Also can be used for a pizza crust.<br /><br />My wife is a celiac so we both enjoy this either as bread sticks or pizza crust.,"[-0.0029680319130420685, -0.006028675008565187...",0.510965
"Versatile Mix;/ This mix makes a good bread or can also be used to make pop overs which are nearly the same as those made with wheat. My wife is a celiac so we use this for bread, popovers and pizza crusts.","[0.0085586654022336, 0.014300554990768433, 0.0...",0.503675
"Best Biscuit Flour you can get!;/ I have use many ready made biscuit flours as well as many attempts to make my own, this is by far the best! Just wish I could get it at my local market!","[-0.0002999216958414763, -0.01683962531387806,...",0.492648
"made a great waffle...;/ Bought this with my new Oster Belgium waffle maker, and had the best waffle ever. Came out very light and crisp. So easy, didn't add oil or anything. I like that it has 4g of fiber per waffle. I always try to get high fiber bread and carb items.","[-0.02342243678867817, -0.01209971308708191, -...",0.449817
Tasty and easy to use.;/ Everyone that dips with this loves it! So easy to use! Olive oil and tasty bread is all you need.,"[0.02885468676686287, -0.014232379384338856, -...",0.443517


In [29]:
# 두 벡터
a = np.array([1, 2, 3])
b = np.array([1, 2, 3])

# 코사인 유사도 계산
result = cosine_similarity([a], [b])
print(result)             # shape = (1, 1)
print(result.shape)       # (1, 1)
print(result[0, 0])       # 1.0
print(type(result[0, 0])) # <class 'numpy.float64'>

[[1.]]
(1, 1)
1.0
<class 'numpy.float64'>


In [33]:
embed_df['embedding']

,embedding
combined,
where does one start...and stop... with a treat like this;/ Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.029458723962306976, -0.021374186500906944, ..."
"Arrived in pieces;/ Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.007512290962040424, 0.03292679786682129, -0..."
"It isn't blanc mange, but isn't bad . . .;/ I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[3.479194492683746e-05, 0.004105150233954191, ..."
"These also have SALT and it's not sea salt.;/ I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.016487499698996544, 0.01567600667476654, -..."
Happy with the product;/ My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[0.0015896736877039075, -0.07027246803045273, ..."
...,...
Delicious!;/ I have ordered these raisins multiple times. They are always great and arrive timely. I can't go back to store bought chocolate covered raisins now! Love this product.,"[0.01768818311393261, -0.05728518217802048, -0..."
Good Training Treat;/ My dog will come in from outside when I am training her and look at the cupboard waiting for her treat. When I use the clicker training method she comes because she knows she has something special.,"[-0.02626226097345352, -0.019168008118867874, ..."
Jamica Me Crazy Coffee;/ Wolfgang Puck's Jamaica Me Crazy is that wonderful blend of island flavors in a coffee. Have loved it from the first time tasting. Great product.,"[-0.027549518272280693, -0.06026146933436394, ..."
